In [ ]:
!pip install -q torch pypdf transformers sentence-transformers langchain langchain_community langchain-huggingface langchain_experimental langchain_chroma langchainhub streamlit accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 833.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.8 MB/s 

In [ ]:
import wikipediaapi

# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia('cmbhatt99@gmail.com', 'en')

def scrape_page(page_name):
    """
    This function takes a page name and retrieves its full text.
    """
    # Get the page object
    page = wiki_wiki.page(page_name)

    # Check if the page exists
    if not page.exists():
        return None

    # Retrieve the full text of the page
    text = page.text
    # Clean the text by removing newline characters
    text_cleaned = text.replace("\n", " ")

    return text_cleaned

def scrape_category_pages(category_name):
    """
    This function takes in each element of list_of_categories and visits all the pages in it to scrape their full text.
    """
    summaries = []

    # Get the category page
    category_page = wiki_wiki.page("Category:" + category_name)

    # Iterate over the category members
    for member in category_page.categorymembers.values():
        # Check if the member is a page
        if member.ns == wikipediaapi.Namespace.MAIN:
            # Get the page object
            page = wiki_wiki.page(member.title)
            # Retrieve the summary of the page
            summary = page.summary
            # Clean the summary by removing newline characters
            summary_cleaned = summary.replace("\n", " ")
            # Add the cleaned summary to the list
            summaries.append(summary_cleaned)

    return summaries

# List of pages and categories to scrape
list_of_pages = [
    'History of the Israeli–Palestinian conflict',
    '1948 Arab–Israeli War',
    'Six-Day War',
    'Yom Kippur War',
    'Arab–Israeli conflict',
    'Israel–Hamas war',
    'Israeli–Palestinian conflict',
    'Al-Shifa Hospital siege',
    'Gaza–Israel conflict',
    'Nakba',
    '1948 Palestine war'
]

list_of_categories = ['History of the Middle East', 'History of the Jews in the Middle East', 'Israel–Hamas war', 'Timeline of the Israel–Hamas war']

# Collect full texts from all pages
knowledge_base = []
for page in list_of_pages:
    full_text = scrape_page(page)
    if full_text:
        knowledge_base.append(full_text + "\n\n")

# Collect summaries from all categories
for category in list_of_categories:
    category_summaries = scrape_category_pages(category)
    knowledge_base.extend([summary + "\n\n" for summary in category_summaries])

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/sodapdf-converted (1).pdf")
knowledge_base = loader.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
knowledge_base_split = text_splitter.split_documents(knowledge_base)
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from re import search
from langchain.vectorstores import Chroma
db = Chroma.from_documents(knowledge_base_split, embeddings)
my_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
!pip install huggingface_hub
from huggingface_hub import login

# This will prompt you to enter your Hugging Face token
login()


In [ ]:
from langchain_huggingface import ChatHuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)



tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-2-2b-it",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=175,
        do_sample=False,
        repetition_penalty=1.03
    ),
    model_kwargs={"quantization_config": nf4_config, "temperature": 0.1},
)



`low_cpu_mem_usage` was None, now set to True since model is quantized.


model-00001-of-00002.safetensors:   8%|7         | 388M/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert assistant. Use only 2 sentences to summarize the context. Here is the context: {context}"),
        ("human", "{user_input}"),
    ]
)


In [ ]:
chat_model = ChatHuggingFace(llm=llm, chat_template = chat_template, streaming = True)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory


conversation_with_summary = ConversationChain(
    llm=chat_model,
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [ ]:
response=conversation_with_summary.predict(input="What was the yom kippur war?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1364: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What was the yom kippur war?
AI:

> Finished chain.


In [ ]:
response

"<bos><start_of_turn>user\nThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: What was the yom kippur war?\nAI:<end_of_turn>\n<start_of_turn>model\nThe Yom Kippur War was a major conflict that took place in the Middle East in 1973.  Here's a breakdown:\n\n* **Date:** It began on October 6, 1973, during the Jewish holiday of Yom Kippur, which is a day of fasting and reflection.\n* **Who fought:**  The main combatants were Israel and its Arab neighbors, including Egypt and Syria.\n* **Why it happened:**  The war was triggered by a surprise attack by Egypt and Syria on Israel on Yom Kippur.  The Egyptians had been planning this for months, and the Syrians had been preparing as well. \n* **Key events:**  The initial attacks were successful, but Israel managed to repel them with 

In [ ]:
def models_response(query):
  answer = ""
  documents = my_retriever.invoke(query)
  for document in documents:
    answer += document.page_content.replace("\n"," ") + "\n"
  return answer

In [ ]:
chat_history=[]

In [ ]:
query = "What was the yom kippur war?"
answer = models_response(query)

prompt = chat_template.format(context=answer, user_input=query)
response = chat_model.invoke(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1364: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and 

In [ ]:
response

AIMessage(content='<bos><start_of_turn>user\nSystem: You are an expert assistant. You must answer questions using only the provided context and ensure that your answers are complete and informative. Do not provide answers or information that is not mentioned in the context. Here is the context: in Ma\'alot killing twenty-two children.In 1973 The Syrian and Egyptian armies launched the Yom Kippur  War, a well-planned surprise attack against Israel. The Egyptians and Syrians advanced during the first  24–48 hours, after which momentum began to swing in Israel\'s favor. Eventually a Disengagement of  Forces agreement was signed between the parties and a ceasefire took effect that ended the war. The  Yom Kippur War paved the way for the Camp David Accords in 1978, which set a precedent for future  peace negotiations.In 1974 the PLO adopted the Ten Point Program, which called for the establishment of a national authority "over every part of Palestinian territory that is liberated" with the 

In [ ]:
split_content = response.content.split("<|assistant|>\n")

# Extract and print the answer
if len(split_content) > 1:
    answer = split_content[1].strip()
    print("Extracted Answer:", answer)
else:
    print("No answer found")

Extracted Answer: In the context of the yom kippur war, the yom kippur war (1973) refers to the surprise attack by Egypt and Syria on Israel on October 6, 1973, which led to the Israeli military being caught off guard and unprepared. The war was fought indirectly, with indirect confrontation between the US and the Soviet Union. The Yom Kippur War accommodated indirect confrontation between the US and the Soviet Union, and the Israeli Army mobilized with their Blue-White Alert, in response to both the warnings and exer


In [ ]:

import gradio as gr
def chat_with_bot(message):
    response = respond(message)
    return response

# Set up Gradio interface
gr.ChatInterface(
    chat_with_bot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me anything about the conflict", container=False, scale=7),
    title="Chinmay's Israel-Palestine Bot",
    description="Abstractive question answering system",
    theme="soft",
).launch(debug=True)